# Lab 2: River Discharge, Continued
---

Today we will continue where we left off from Lab 1, calculating the discharge of a river. You'll want to keep your Lab 1 notebook open and handy, since we'll use some results from it.

## Submission

Please save this file (with your additional answer cells included — make sure they run!), download it (**File $\rightarrow$ Download .ipynb**), and upload to Canvas.

## Preparation

First, let's read in the `riverdata.npz` file again.


In [ ]:
import numpy as np
    
riverdata = np.load('riverdata.npz')

x = riverdata['x']
rdepth = riverdata['rdepth']
surfvel = riverdata['surfvel']

## Numerically Calculating Average Velocities

For calculating average velocities it is easiest to think of each of each of our measurement data as being one column of water 1 meter wide with a height equal to the water depth and a surface velocity equal to the measured velocity. We want to calculate the average velocity for each column separately, because the column of water has different heights for different positions $x$ across the river, short columns near the edges and tall columns near the middle. We have two ways of calculating this. We can use the analytical expression you derived last week, or we can numerically integrate the velocity profile (which we might need to do if the profile $v(z)$ doesn't have a nice equation like the one provided in last week's lab).

In this lab we'll do it both ways to see the difference. But first, we need to know what $k$ (coefficient describing water properties) is!

## <span style="color:green">Question 5</span>

Using the expression for $v(z)$ provided in the last lab and the known velocity points ($z = 0$ at the bottom of the channel and $z = H$ at the surface), derive an expression for $k$ in terms of the surface velocity and the water depth (our known parameters).

## Using the analytical average equation

Of course if we can calculate the average of a profile analytically, that is easiest. Using your derived expression for $k$ and for the average velocity $\bar{v}$, we can calculate the average velocity for each column of water. But we want to take advantage of NumPy's ability to work with arrays, so rather than doing a separate calculation for each column, or doing a `for` loop, we will do it using simpler manipulations of matrices. NumPy can add, multiply, and divide matrices (in NumPy parlance, both matrices and vectors are called "arrays") that are the same size element-by-element very quickly. Here are some examples:

1. Define some vectors:

In [ ]:
P = np.arange(1, 101)  # This creates an array of integers from 1 to 100
R = np.random.random_sample(P.shape)  # Make a vector of random values with the same dimensions as P

2. To multiply element-by-element, use `*`. Try also `/` and `**2` (`**` means "to the power of" in Python). These are all operating element-by-element.

In [ ]:
# Experiment here!

Now that you are familiar with manipulating NumPy arrays, write the code to calculate the discharge:

1. Use your data `x`, `surfvel`, and `rdepth` to write a one-line expression for $k$.

2. Now, with $k$ defined, calculate the average velocity, using your theoretical expression.

3. To calculate discharge from each column of water, you can now write an expression for $Q = Av$ for each column of water.

4. Finally, you can calculate the total discharge by adding the discharge from all the columns.

## <span style="color:green">Question 6</span>

In a **new code cell below**, include your code which calculates the total discharge analytically. (Either have your code print the value, or include that output value in a comment.)

## Fully Numeric Method

If you can't analytically integrate your velocity profile, then you might need to do it fully numerically. This also allows you to look at the velocity pattern a bit more. Again, we want to take advantage of NumPy's strengths and avoid using `for` loops.

1. The first thing we need to do is work in 2D matrices instead of the 1D vectors for the horizontal and vertical position. Each column of water has a different depth, but we want to do our calculations most efficiently, so we create a grid of relative $z$ positions:

In [ ]:
zrel = np.arange(0, 1 + 0.05, 0.05)
zrel_rep = np.tile(zrel, (x.size, 1)).T

This created an array called `zrel` with 21 evenly spaced $z$ values between 0 and 1. Then it takes that and repeats it ("tiles" it) for every $x$ position across the river. In other words, we are now working on a grid where we have 21 depth points and 51 horizontal points, so it is a 21 $\times$ 51 matrix.

2. We need to convert these relative depths into real depths, because the real depths define our velocity profiles. Here we just change variable names to something that is easier to picture, and we take the `zmax` and repeat it 21 times to make it also a 21 $\times$ 51 matrix. Now we can multiply the two matrices because they are both 21 $\times$ 51. This gives us a matrix `z` that gives actual depth and horizontal position.

In [ ]:
zmax = rdepth
zmax_rep = np.tile(zmax, (zrel.size, 1))
z = zrel_rep * zmax_rep

3. If we want to look at `z` to see if it makes sense, an easy quick look at a matrix uses the below command. Remember that the color scale here denotes depth.

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(z);

#let's add a color bar and some labels for good measure.
plt.xlabel('Distance across channel [m]')
plt.ylabel('Depth slice')
plt.colorbar(orientation='horizontal').set_label('water depth [m]');

4. Now that we have a `z` matrix, we need a similar velocity matrix — but to get there, we need the $k$ you calculated earlier, but we need to turn it into a matrix by repeating the rows:

In [ ]:
 k_rep = np.tile(k, (len(zrel), 1))

5. Now you can calculate the velocity profile for each column of water using the expression for $v(z)$ from last lab:

In [ ]:
vel = k_rep * z * (2 * zmax_rep - z)

6. Now you can plot a single column's velocity profile like this. You can change the `COLUMN_INDEX` to view different columns.

In [ ]:
fig, ax = plt.subplots()

COLUMN_INDEX = 34

ax.plot(vel[:, COLUMN_INDEX], z[:, COLUMN_INDEX])
ax.set_xlabel('Velocity (m/s)')
ax.set_ylabel('Distance from Channel Bottom (m)')

plt.show()

7. You can also plot all of them together (but first we need an `x_rep` similar to our `z_rep`).

In [ ]:
x_rep = np.tile(x, (len(zrel), 1))

fig, ax = plt.subplots()

cm = ax.pcolormesh(x_rep, z - zmax_rep, vel)

ax.set_xlabel('Distance Across Channel (m)')
ax.set_ylabel('Distance downward from Water Surface (m)')

fig.colorbar(cm, label='Velocity (m/s)')

plt.show()

8. Now you can numerically calculate the average velocity for each column in one line. The [`scipy.integrate.cumtrapz()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumtrapz.html) function will work on the first dimension of an array, therefore all that it takes to numerically integrate is:

In [ ]:
from scipy.integrate import cumtrapz

integrated_vel = cumtrapz(vel, axis=0, initial=0)

The average velocity, however, requires us to divide by the number of grid points:

In [ ]:
avg_vel = integrated_vel[-1, :] / len(zrel)

Now the discharge for each column of water is:

In [ ]:
col_discharge = avg_vel * rdepth * (x[1] - x[0])  # (x[1] - x[0]) is dx

And the total discharge is:

In [ ]:
total_discharge = sum(col_discharge)

## <span style="color:green">Question 7</span>

In a **new text cell below**, compare the total discharge from the numerical version immediately above to the analytical result you obtained in Question 6.

## <span style="color:green">Question 8</span>

In a **new code cell below**, write the code to plot the discharge of each column as a function of distance across the river for both the analytical version and the numeric version for comparison. Make sure your code shows the plot when run!